In [1]:
%bazel "@PythonKit//:PythonKit"
%bazel "@SwiftCrypto//:Crypto"
%bazel "//mujoco:jupyter"

Installing packages:
|-@PythonKit//:PythonKit
|-@SwiftCrypto//:Crypto
|-//mujoco:jupyter
Working in: /home/liu/Dropbox/workspace/sim2real/.ibzlnb/ivXSD
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-fastbuild/bin/mujoco/MuJoCoJupyterExtensions.swiftmodule
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-fastbuild/bin/external/PythonKit/PythonKit.swiftmodule
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-fastbuild/bin/external/SwiftCrypto/Crypto.swiftmodule
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-fastbuild/bin/external/SwiftNIO/NIO.swiftmodule
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-fastbuild/bin/external/SwiftNIO/NIOConcurrencyHelpers.swiftmodule
swiftmodule: 

In [2]:
import PythonKit

In [3]:
%include "EnableJupyterDisplay.swift"

In [4]:
import MuJoCo
import MuJoCoJupyterExtensions

In [5]:
let simulate = Simulate(width: 800, height: 600, title: "simulate")
var model = try MjModel(fromXMLPath: "/home/liu/workspace/s4nnc/gym/assets/ant.xml")
var data = model.makeData()
simulate.use(model: model, data: &data)

In [6]:
let httpRenderServer = HTTPRenderServer(simulate, maxWidth: 800, maxHeight: 600, canResize: false)
let channel = try httpRenderServer.bind(host: "192.168.86.5", port: 12345).wait()

In [7]:
simulate.makeContext(hidden: true)
let cpugenesis = GLContext.time
let simgenesis = data.time
var cpusync = cpugenesis
var simsync = simgenesis

In [8]:
JupyterDisplay.display(html: httpRenderServer.html)

In [9]:
while true {
    model.step(data: &data)
    simsync = data.time
    while simsync - simgenesis >= cpusync - cpugenesis {
        simulate.yield()
        cpusync = GLContext.time
    }
}

: 